In [ ]:
# -*- coding: utf-8 -*-
# 1.导入包以及设置随机种子
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from math import sqrt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

import random
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
# 2.以类的方式定义超参数
class argparse():
    pass

args = argparse()

# 固定参数

args.patience = 30      #early stopping相关
args.dimension = 4

# 可优化参数
args.epochs =300
args.BTACH_SIZE =512     
args.data = pd.read_csv('./rainfall_traindata.csv')
args.delay =72
args.device= [torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),]
args.layers=2

args.data.set_index('time', inplace=True)

In [ ]:
#定义数据集函数  
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        return len(self.features)
    
def build_databbase(sequence_length,delay,data):
    data_ = []
    for i in range(len(data) - sequence_length - delay):
        data_.append(data.iloc[i: i + sequence_length + delay])

    data_ = np.array([df.values for df in data_])
    np.random.shuffle(data_)
    x = data_[:, :-delay, :]
    y = data_[:, -1, 0]
    #y = data_[:,-delay:, 0]

    x = x.astype(np.float32)
    y = y.astype(np.float32)
   
    #根据6：2：2的比例划分训练集和测试集
    split_boundary = int(data_.shape[0] * 0.8)

    train_x = x[: split_boundary]
    test_x = x[split_boundary:]

    train_y = y[: split_boundary]
    test_y = y[split_boundary:]

    mean = train_x.mean(axis=0)
    std = train_x.std(axis=0)
    train_x = (train_x - mean)/std
    test_x = (test_x - mean)/std
    train_ds = Mydataset(train_x, train_y)
    test_ds = Mydataset(test_x, test_y)

    train_dl = torch.utils.data.DataLoader(
                                           train_ds,
                                           batch_size=args.BTACH_SIZE,
                                           shuffle=True
    )
    test_dl = torch.utils.data.DataLoader(
                                           test_ds,
                                           batch_size=args.BTACH_SIZE
    )
    return train_dl,test_dl


In [ ]:
# 2.定义模型
class Net(nn.Module):
    def __init__(self, hidden_size):
        super(Net, self).__init__()
        self.rnn = nn.LSTM(args.dimension, 
                           hidden_size,
                           args.layers,
                           batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, inputs):
       
        s_o,_ = self.rnn(inputs)
        s_o = s_o[:, -1, :]
        x = F.dropout(F.relu(self.fc1(s_o)),0.2,training=True)
        x = F.relu(self.fc2(x))
        return torch.squeeze(x)

In [ ]:
# 3.定义early stopping
class EarlyStopping():
    def __init__(self,patience=7,verbose=False,delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        
    def __call__(self,val_loss,model,path):
        #print("val_loss={}".format(val_loss))
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss,model,path)
        elif score < self.best_score+self.delta:
            self.counter+=1
            #print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter>=self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss,model,path)
            self.counter = 0
            
    def save_checkpoint(self,val_loss,model,path):
        if self.verbose:
            print(
                f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
            torch.save(model.state_dict(), path+'/'+'model_earlystopping.pth')
        self.val_loss_min = val_loss

In [ ]:
loss_fn = nn.MSELoss()

In [ ]:
# 4.定义训练函数
def fit(epoch, model, trainloader, testloader,optimizer):
    total = 0
    running_loss = 0
    
    model.train()
    for x, y in trainloader:
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x)
        loss = loss_fn(y_pred, y) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            total += y.size(0)
            running_loss += loss.item()
#    exp_lr_scheduler.step()
    epoch_loss = running_loss / len(trainloader.dataset)
        
        
    test_total = 0
    test_running_loss = 0 
    
    model.eval()
    with torch.no_grad():
        for x, y in testloader:
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    
        
    #print('epoch: ', epoch, 
    #      'loss： ', round(epoch_loss, 3),
    #      'test_loss： ', round(epoch_test_loss, 3),
    #         )
    
        
    return epoch_loss, epoch_test_loss

In [ ]:
# 10.PSO寻优算法
#
# 10.1 define PSO
class PSO(object):
    def __init__(self,particle_num,particle_dim,iter_num,c1,c2,w,max_value,min_value):
        '''参数初始化
        particle_num(int):粒子群的粒子数量
        particle_dim(int):粒子维度，对应待寻优参数的个数
        iter_num(int):最大迭代次数
        c1(float):局部学习因子，表示粒子移动到该粒子历史最优位置(pbest)的加速项的权重
        c2(float):全局学习因子，表示粒子移动到所有粒子最优位置(gbest)的加速项的权重
        w(float):惯性因子，表示粒子之前运动方向在本次方向上的惯性
        max_value(float):参数的最大值
        min_value(float):参数的最小值
        '''
        self.particle_num = particle_num ##
        self.particle_dim = particle_dim ##本例中使用2个，分别对应[hidden_size, learning_rate]
        self.iter_num = iter_num
        self.c1 = c1  ##通常设为2.0
        self.c2 = c2  ##通常设为2.0
        self.w = w    
        self.max_value = max_value ##200
        self.min_value = min_value ##0
        
        
# 10.2 initial particle swarm
    def swarm_origin(self):
        '''粒子群初始化
        input:self(object):PSO类
        output:particle_loc(list):粒子群位置列表
               particle_dir(list):粒子群方向列表
        '''
        particle_loc = []
        particle_dir = []
        for i in range(self.particle_num):
            tmp1 = []
            tmp2 = []
            for j in range(self.particle_dim):
                a = random.random()
                b = random.random()
                tmp1.append(a * (self.max_value[j] - self.min_value[j]) + self.min_value[j])
                tmp2.append(b)
            particle_loc.append(tmp1)
            particle_dir.append(tmp2)
        
        return particle_loc,particle_dir

# 10.3 计算适应度函数数值列表;初始化pbest_parameters和gbest_parameter   
    def fitness(self,particle_loc):
        '''计算适应度函数值
        input:self(object):PSO类
              particle_loc(list):粒子群位置列表
        output:fitness_value(list):适应度函数值列表
        '''
        train_loss = []
        test_loss = []
        train_epochs_loss = []
        valid_epochs_loss = []
        fitness_value = []
        ### 1.适应度函数为loss值
        for i in range(self.particle_num):
            print(f"{i}/{self.particle_num} particle.")
            #构建数据集
            train_dl,test_dl=build_databbase(int(particle_loc[i][2]),args.delay,args.data)#时间步长
            criterion = torch.nn.MSELoss()
            lst = Net(int(particle_loc[i][0])+2)#hidden_size值
            opti = torch.optim.Adam(lst.parameters(), lr=particle_loc[i][1])
            criterion = torch.nn.MSELoss()
            early_stopping = EarlyStopping(patience=args.patience)
            print(particle_loc[i][0],particle_loc[i][1],particle_loc[i][2])
            for epoch in range(args.epochs):
                lst.to('cuda')
                epoch_loss, epoch_test_loss = fit(epoch,
                                                  lst,
                                                  train_dl,
                                                  test_dl,
                                                  opti)
    
                train_loss.append(epoch_loss)
                test_loss.append(epoch_test_loss)
                #==================early stopping======================
                early_stopping(test_loss[-1],model=lst,path='./models')
                if early_stopping.early_stop:
                    print("Early stopping")
                    break
            # rbf_svm = svm.SVC(kernel = 'rbf', C = particle_loc[i][0], gamma = particle_loc[i][1])
            # cv_scores = cross_validation.cross_val_score(rbf_svm,trainX,trainY,cv =3,scoring = 'accuracy')
            print("val_loss={}".format(test_loss[-1]))
            fitness_value.append(-test_loss[-1])
        ### 2. 当前粒子群最优适应度函数值和对应的参数
        current_fitness = -1000.0
        current_parameter = []
        for i in range(self.particle_num):
            if current_fitness < fitness_value[i]:
                current_fitness = fitness_value[i]
                current_parameter = particle_loc[i]

        return fitness_value,current_fitness,current_parameter 
        

## 10.4  粒子位置更新 
    def updata(self,particle_loc,particle_dir,gbest_parameter,pbest_parameters):
        '''粒子群位置更新
        input:self(object):PSO类
              particle_loc(list):粒子群位置列表
              particle_dir(list):粒子群方向列表
              gbest_parameter(list):全局最优参数
              pbest_parameters(list):每个粒子的历史最优值
        output:particle_loc(list):新的粒子群位置列表
               particle_dir(list):新的粒子群方向列表
        '''
        ## 1.计算新的量子群方向和粒子群位置
        for i in range(self.particle_num): 
            a1 = [x * self.w for x in particle_dir[i]]
            a2 = [y * self.c1 * random.random() for y in list(np.array(pbest_parameters[i]) - np.array(particle_loc[i]))]
            a3 = [z * self.c2 * random.random() for z in list(np.array(gbest_parameter) - np.array(particle_dir[i]))]
            particle_dir[i] = list(np.array(a1) + np.array(a2) + np.array(a3))
            # particle_dir[i] = self.w * particle_dir[i] + self.c1 * random.random() * (pbest_parameters[i] - particle_loc[i]) + self.c2 * random.random() * (gbest_parameter - particle_dir[i])
            particle_loc[i] = list(np.array(particle_loc[i]) + np.array(particle_dir[i]))
            
        ## 2.将更新后的量子位置参数固定在[min_value,max_value]内 
        ### 2.1 每个参数的取值列表
        parameter_list = []
        for i in range(self.particle_dim):
            tmp1 = []
            for j in range(self.particle_num):
                tmp1.append(particle_loc[j][i])
            parameter_list.append(tmp1)
        ### 2.2 每个参数取值的最大值、最小值、平均值   
        value = []
        for i in range(self.particle_dim):
            tmp2 = []
            tmp2.append(max(parameter_list[i]))
            tmp2.append(min(parameter_list[i]))
            value.append(tmp2)
        
        for i in range(self.particle_num):
            for j in range(self.particle_dim):
                particle_loc[i][j] = (particle_loc[i][j] - value[j][1])/(value[j][0] - value[j][1]) * (self.max_value[j] - self.min_value[j]) + self.min_value[j]
                
        return particle_loc,particle_dir

## 10.5 画出适应度函数值变化图
    def plot(self,results):
        '''画图
        '''
        X = []
        Y = []
        for i in range(self.iter_num):
            X.append(i + 1)
            Y.append(results[i])
        plt.plot(X,Y)
        plt.xlabel('Number of iteration',size = 15)
        plt.ylabel('Value of CV',size = 15)
        plt.title('PSO_RBF_SVM parameter optimization')
        plt.savefig("rainpso.png",dpi=800,bbox_inches = 'tight')
        plt.show() 
        
## 10.6 主函数        
    def main(self):
        '''主函数
        '''
        results = []
        best_fitness = -1000.0 
        ## 1、粒子群初始化
        particle_loc,particle_dir = self.swarm_origin()
        ## 2、初始化gbest_parameter、pbest_parameters、fitness_value列表
        ### 2.1 gbest_parameter
        gbest_parameter = []
        for i in range(self.particle_dim):
            gbest_parameter.append(-1000)
        ### 2.2 pbest_parameters
        pbest_parameters = []
        for i in range(self.particle_num):
            tmp1 = []
            for j in range(self.particle_dim):
                tmp1.append(-1000)
            pbest_parameters.append(tmp1)
        ### 2.3 fitness_value
        fitness_value = []
        for i in range(self.particle_num):
            fitness_value.append(-1000)
    
        ## 3.迭代
        for i in range(self.iter_num):
            print(f"{i}/{self.iter_num} iter of current particle.")
            ### 3.1 计算当前适应度函数值列表
            current_fitness_value,current_best_fitness,current_best_parameter = self.fitness(particle_loc)
            ### 3.2 求当前的gbest_parameter、pbest_parameters和best_fitness
            for j in range(self.particle_num):
                if current_fitness_value[j] > fitness_value[j]:
                    pbest_parameters[j] = particle_loc[j]
            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                gbest_parameter = current_best_parameter
            
            print('iteration is :',i+1,';Best parameters:',gbest_parameter,';Best fitness(loss)',best_fitness)
            results.append(best_fitness)
            ### 3.3 更新fitness_value
            fitness_value = current_fitness_value
            ### 3.4 更新粒子群
            particle_loc,particle_dir = self.updata(particle_loc,particle_dir,gbest_parameter,pbest_parameters)
        ## 4.结果展示
        results.sort()
        self.plot(results)
        print('Final parameters are :',gbest_parameter)
        return gbest_parameter

In [ ]:
pso = PSO(particle_num=20,particle_dim=3,iter_num=50,c1=2,c2=2,w=0.8,max_value=[80,0.01,300],min_value=[5,0.0001,96])

In [ ]:
gbest_parameter=pso.main()

In [ ]:
#用最佳参数训练模型

train_dl,test_dl,test_y=build_databbase(gbest_parameter[2],args.delay,args.data)

model = Net(gbest_parameter[0])
if torch.cuda.is_available():
    model.to('cuda')
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=gbest_parameter[1])


early_stopping = EarlyStopping(patience=args.patience)


train_loss = []
test_loss = []
start_time = time.time()

for epoch in range(args.epochs):
    epoch_loss, epoch_test_loss = fit(epoch,
                                      model,
                                      train_dl,
                                      test_dl)

    train_loss.append(epoch_loss)
    test_loss.append(epoch_test_loss)
    #==================early stopping======================
    early_stopping(test_loss[-1],model=model,path='./models')
    if early_stopping.early_stop:
        print("Early stopping")
        break

end_time = time.time()
print("耗时: {:.2f}秒".format(end_time - start_time))



In [ ]:
#保存模型
torch.save(model, "./models/PSO_lstm_model.pth")